Data collection from 3 Hungarian property website
The collection runs once a week manually

In [1]:
import requests
#from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
import urllib3
import warnings
warnings.filterwarnings('ignore')
import datetime as dtime
import time

#### Simple request

In [2]:
#ingatlan.hu, one page
def simpleget(url):
    '''
    gets simple one page website from the URL
    
    returns body tag
    '''
    r=requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'})
    
    page = BeautifulSoup(r.content, 'lxml')
    body = page.body
    return body


#### Multi-page search retrieval

In [3]:
#ingatlan.com, multi-page
def inner_multi_get(url, n):
    '''
    gets multipages session based webpages like search results specialized to ingatlan.com site
    
    url for URL
    n for number of pages to follow
    
    returns html tag named 'main'
    '''
    link = url + str(n)
    print(link)
    http = urllib3.PoolManager(headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'})
    response = http.request('GET', link)
    soup = BeautifulSoup(response.data, 'lxml')
    body = soup.find_all('main')
    
    return body

def multipages(url , v=1):
    '''
    wrapper for inner_multi_get(url, n) to handle returned htmls in one list 
    url for URL,
    v for number of pages to return
    
    inner_multi_get:
    gets multipages session based webpages like search results specialized to ingatlan.com site
    
    returns html tag named 'main'
    '''
    i=1
    while i <= v:
        text = inner_multi_get(url, n=i)
        
        if 'csvtext' in locals():
            csvtext = csvtext + text
        else: csvtext = text
            
        time.sleep(1)
        i += 1
    return csvtext

#### Saving html source data into file

In [4]:
def writeto(csvtext, fnev):
    '''
    write csvtext into file with the given filename (fnev)
    
    returns acknowladgement
    '''
    file = open(fnev, 'w')
    file.write('%s' % csvtext)
    file.close()   
    return print('Data saved as %s' % fnev )

def savetext(csvtext,fnev):
    '''
    wrapper to writeto() to create filename (fnev)
    csvtext saved text
    '''
    
    global fn
    # file name includes day-month timestamp and page number for multi-page retievals
    now = dtime.datetime.now()
    m = [str(now.month) if len(str(now.month)) == 2 else '0' + str(now.month)]
    d = [str(now.day) if len(str(now.day)) == 2 else '0' + str(now.day)]
    y = str(now.year)
    fn = fnev + '_' + d[0] + m[0] + y + '.csv'
    writeto(csvtext,fn)   
    return

#### Check number of saved ads

In [5]:
def check(fn , i):
    '''
    Checks the number of ads saved. based on the saved file.
    html read specialized to surces
    '''
    html = (open(fn, 'r').read())
    l = BeautifulSoup(html, 'lxml')
    if 'ingatlanokHU' in fn:      
        print(len(l.find_all('div', attrs={'class': 'result-row'})))
    
    elif 'otthoncentrum' in fn:  
        print(len(l.find_all('div', attrs={'class': 'estate-list-box'})))

    else:
        a = len(l.find_all('div', attrs={'class': 'listing js-listing listing--cluster-parent js-cluster-parent'}))
        b = len(l.find_all('div', attrs={'class': 'listing js-listing'}))
        print("No of ads in {x} : {y}".format(x= i  , y=a+b))

### Set-up

In [6]:
def retriveall_tofile():
    '''
    specalized setup for the webscrapeing with fixed urls, names
    
    returns saved files, counts of ads per file
    '''
    url1 = 'https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page='
    url1a = 'https://ingatlan.com/szukites/elado+lakas+xi-ker+48-55-m2+ar-szerint?page='
    url1b = 'https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page='
    url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-v-ker:25nm-tol;33nm-ig?record=0&num=100'
    url2a = 'https://ingatlanok.hu/elado/lakas+haz/budapest-xi-ker:48nm-tol;55nm-ig?record=0&num=100'
    url2b = 'https://ingatlanok.hu/elado/lakas+haz/budapest-vi-ker:38nm-tol;55nm-ig?record=0&num=100'
    url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest05-kerulet/hely-id:budapest05-kerulet,/netto-alapterulet:28~33/emelet:1,8'
    url3a = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest11-kerulet/hely-id:budapest11-kerulet,/netto-alapterulet:45~55/emelet:1,8'
    url3b = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest06-kerulet/hely-id:budapest06-kerulet,/netto-alapterulet:38~55/emelet:1,8'
    
    hu = {
        'ingatlanokHU_Vker': url2, 
        'ingatlanokHU_XIker': url2a,
        'ingatlanokHU_VIker': url2b,
        'otthoncentrum_Vker': url3,
        'otthoncentrum_XIker': url3a,
        'otthoncentrum_VIker': url3b
    }
    com = {
        'ingatlanCOM_Vker': url1,
        'ingatlanCOM_XIker': url1a,
        'ingatlanCOM_VIker': url1b,

    }
       
    #one page download
    for i in hu.keys():
        csvtext = simpleget(hu[i])
        savetext(csvtext, i)
        check(fn ,i)
    
    
    #multipage download
    for i in com.keys():
        csvtext = multipages(com[i], v=10)
        savetext(csvtext, i)
        check(fn, i)
        time.sleep(3)
    

#### Runing the show

In [7]:
retriveall_tofile()

Data saved as ingatlanokHU_Vker_23022020.csv
17
Data saved as ingatlanokHU_XIker_23022020.csv
104
Data saved as ingatlanokHU_VIker_23022020.csv
100
Data saved as otthoncentrum_Vker_23022020.csv
3
Data saved as otthoncentrum_XIker_23022020.csv
32
Data saved as otthoncentrum_VIker_23022020.csv
48
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=1
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=2
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=3
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=4
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=5
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=6
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=7
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=8
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=9
https://ingatlan.com/szukites

In [7]:
retriveall_tofile()

Data saved as ingatlanokHU_Vker_16022020.csv
21
Data saved as ingatlanokHU_XIker_16022020.csv
100
Data saved as ingatlanokHU_VIker_16022020.csv
100
Data saved as otthoncentrum_Vker_16022020.csv
4
Data saved as otthoncentrum_XIker_16022020.csv
34
Data saved as otthoncentrum_VIker_16022020.csv
48
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=1
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=2
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=3
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=4
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=5
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=6
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=7
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=8
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=9
https://ingatlan.com/szukites